<h1 align="center">Yleistetyt Lineaariset Mallit 2, Demo 2</h1>

<br> 

In [1]:
import pandas as pd
import numpy as np
import scipy
import statsmodels.api as sm

<h3 align="left">Tehtävä 6</h3>

Tarkastellaan luentomonisteen esimerkin 13. lineaarista regressiomallia

$ mpist_i = \beta_0 \, + \, \beta_1 \, matem_i \, + \, \beta_2 \, sukup_i \, + \, \beta_3 \, koulusij_i \, \\ + \, \beta_4 \, koulualue\_ita_i \, + \,       \beta_5 \, koulualue\_lansi_i \, + \, \beta_6 \, koulualue\_pohj_i \, + \,     \epsilon_i $

Lue aineisto R:ään ja laske käyttäen hyväksi luentomonisteen analyyttisiä tuloksia

- Parametrivektorin β suurimman uskottavuuden estimaatti
-  Mallin keskivirhe $ \mathbf{\hat{\sigma}} $
- Suurimman uskottavuuden estimaattien $ \mathbf{\hat{\beta}} $ keskivirheet
- Mallin selitysaste $ \mathbf{R^2} $

In [2]:
pisa = pd.read_csv("C:/Users/testi/Desktop/excel_files/pisa_demo02.csv", 
                   index_col=0)

In [3]:
pisa.head(3)

,mpist,sukup,SES,koulusij,koulualue,matem,aidink
107,569.254,poika,0.1322,kaupunki,Etela-Suomi,9,7
1261,631.804,poika,1.1144,kaupunki,Etela-Suomi,8,8
381,521.582,tytto,0.4400,kaupunki,Etela-Suomi,7,9


In [4]:
# Luodaan deisgn-matriisi.

mat = pisa["matem"].values

sp = pisa["sukup"].replace({"poika": 0, "tytto": 1}).values

sij = pisa["koulusij"].replace({"kaupunki": 0, "maaseutu": 1}).values

ita = np.where(pisa["koulualue"] == "Ita-Suomi", 1, 0)

lansi = np.where(pisa["koulualue"] == "Lansi-Suomi", 1, 0)

pohjoinen = np.where(pisa["koulualue"] == "Pohjois-Suomi", 1, 0)

intercept = np.ones(200, dtype=int)

X = pd.DataFrame({"intercept": intercept,
                  "mat": mat,
                  "sp": sp,
                  "sij": sij,
                  "ita": ita,
                  "lansi": lansi,
                  "pohjoinen": pohjoinen})

X.index += 1

In [64]:
X.shape

(200, 7)

In [58]:
# Vastemuuttuja
y = pisa["mpist"]

In [63]:
y.shape

(200,)

<br>

**Parametrivektorin β suurimman uskottavuuden estimaatti**

Parametrivektorin $\, \beta \,$ suurimman uskottavuuden estimaatti saadaa optimoimalla (maksimoimalla) logaritminen uskottavuusfunktio $\, \beta \,$:n suhteen, jolloin

\begin{equation}
    \large\hat{\beta} = (X^T X)^{-1} \, X^T y
\end{equation}

In [67]:
betahat = np.linalg.inv(X.values.T @ X.values) @ X.values.T @ y.values
betahat

array([334.61292837,  29.96626779, -17.34266536,  15.10478975,
        -3.57897317, -31.29944545, -11.89215235])

- Huomaa, että values-attribuutti muuttaa Pandasin DataFrame- ja Series-objektit NumPy-taulukoiksi.
- Tämän seurauksena parametrivektorin $\, \beta \,$ SU-estimaatti saadaan NumPy-taulukkona.
- Jos values-attribuutit jätettäisiin pois, saataisiin sama SU-estimaatti, mutta Pandasin Series-objektina.

In [68]:
MLEs = pd.DataFrame({"MLE": betahat},
                     index=["intercept", "mat", "sp", "sij", "ita", "lansi", "pohjoinen"])

In [69]:
MLEs

,MLE
intercept,334.612928
mat,29.966268
sp,-17.342665
sij,15.104790
ita,-3.578973
lansi,-31.299445
pohjoinen,-11.892152


Verrataan näitä vaikkapa Statsmodels-kirjastosta löytyvään valmiiseen funktioon.

In [80]:
m = sm.OLS(y.values, X.values).fit()
np.round(m.params, 6)

array([334.612928,  29.966268, -17.342665,  15.10479 ,  -3.578973,
       -31.299445, -11.892152])

- Nämä ovat identtiset johtuen siitä, että kun X on kääntyvä, niin statsmodels-kirjaston OLS-funktio etsii ratkaisun yllä määritellyn normaaliyhtälön avulla (joka on siis parametrivektorin $\, \beta \,$ suurimman uskottavuuden estimaatti).

<br>

**Mallin keskivirhe** $ \mathbf{\hat{\sigma}} \, $ on

$$ \mathbf{\hat{\sigma}} = \sqrt{\frac{(y - X\hat{\beta})^T \, (y - X\hat{\beta})}{n - p - 1}}. $$

Mallin keskivirheen laskemiseksi tarvitaan jäännökset

$$ \large\mathbf{\hat{\epsilon}} = y - \hat{y} $$

missä $ \mathbf{\hat{y}} $ ovat mallin sovitteet, jotka saadaan kaavalla

$$ \large\mathbf{\hat{y}} = X\hat{\beta} $$

missä X on design-matriisi ja $ \mathbf{\hat{\beta}} $ on parametrivektori.

- **Huomaa**, että keskivirheen kaavassa osoittajassa on residuaalivektorin transpoosi kertaa residuaalivektori.
- p = parametrivektorin $ \hat{\beta} $ parametrien (kertoimien) lukumäärä. **Huomaa**, että tähän ei lasketa intercept-termiä mukaan!

In [14]:
resid = y - np.matmul(X, betahat)

In [15]:
n = len(X)

In [16]:
p = betahat.shape[0] - 1

In [17]:
sigmahat = np.sqrt((np.matmul(resid.T, resid)) / (n - p - 1))

In [18]:
print(f"Mallin keskivirhe on: {sigmahat}")

Mallin keskivirhe on: 63.4001728494971


<br>

**Suurimman uskottavuuden estimaattien $ \mathbf{\hat{\beta}} $ keskivirheet**

Parametrien $ \mathbf{\hat{\beta_j}} $ keskivirheet ovat

$$ \sqrt{\widehat{Var}(\hat{\beta}_j}) = \sqrt{[\widehat{Cov}(\hat{\beta})}]_{jj} := se(\hat{\beta}_j), $$

$$ missä \quad \widehat{Cov}(\hat{\beta}) = \sigma^2(X^TX)^{-1}. $$

In [81]:
covbeta = sigmahat**2 * np.linalg.inv(X.values.T @ X.values)

In [82]:
sdbeta = np.sqrt(np.diagonal(covbeta))

In [83]:
pd.DataFrame({"Standard error": np.round(sdbeta, 2)},
              index=["intercept", "mat", "sp", "sij", "ita", "lansi", "pohjoinen"])

,Standard error
intercept,24.68
mat,3.27
sp,9.03
sij,11.81
ita,15.26
lansi,10.89
pohjoinen,14.52


<br>

**Mallin selitysaste** $ \mathbf{R^2} $

$$ SST = \sum_{i=1}^n (y_i - \bar{y})^2 $$

$$ SSR = \sum_{i=1}^n (\hat{y}_i - \bar{y})^2 $$

$$ SSE = \sum_{i=1}^n (y_i - \hat{y}_i)^2 $$

$$ \hat{y} = X\hat{\beta} $$

- SST on vasteen kokonaisvaihtelu
- SSR on regressiomallin selittämä vaihtelu
- SSE on selittämätön vaihtelu (jäännösvaihtelu)

$$ R^2 = \frac{SSR}{SST} = 1- \frac{SSE}{SST} $$

In [23]:
SST = sum((y - np.mean(y))**2)

In [24]:
yhat = np.matmul(X_arr, betahat)

In [25]:
SSE = sum((y - yhat)**2)

In [26]:
Rsquared = 1 - (SSE / SST)

In [27]:
print(f"Mallin selitysaste on: {Rsquared}")

Mallin selitysaste on: 0.33300709002407625


<br>

<br>

<h3 align="left">Tehtävä 7</h3>

Testaa t-testin avulla edellisen tehtävän lineaarisen mallin tapauksessa sukupuoleen liittyvää regressiokerrointa, eli nollahypoteesiä $ H_0 : \beta_2 = 0. $

t-testisuure lineaarisen mallin yksittäiselle regressiokertoimelle saadaan kaavalla

$$ t = \frac{\hat{\beta}_j - \beta_j}{se(\hat{\beta}_j)}. $$

Huomaa, että $ \mathbf{\beta_j} $ on nollahypoteesin mukainen regressiokerroin (nolla) eli osoittajaan jää ainoastaan testattava regressiokerroin. 

In [28]:
t = betahat[2] / sdbeta[2]

In [29]:
print(f"Testisuure on: {t}")

Testisuure on: -1.9206433364970608


In [30]:
p_value = 2 * (1 - scipy.stats.t.cdf(np.abs(t), n-p-1))

In [31]:
print(f"p-arvo on: {p_value}")

p-arvo on: 0.056250599851747296


- Huomaa, että t-testiä voidaan käyttää ainoastaan, kun testataan yksittäisen regressiokertoimen merkitsevyyttä.
- Jos halutaan testata useampaa regressiokerrointa samanaikaisesti (esim. $ H_0 : \beta_4 = \beta_5 = \beta_6 = 0 $), voidaan käyttää **F-testiä.**

<br>

<br>

**Tehtävä 8**

Luo oma funktio F-testistä ja testaa sen avulla annettuja hypoteeseja.

In [32]:
def F_test(K, X, beta, sigma):
    q = np.linalg.matrix_rank(np.linalg.qr(K)[0])
    n = X.shape[0]
    p = X.shape[1]
    F = (K.T @ beta).T @ np.linalg.inv(K.T @ np.linalg.inv(X.T @ X) @ K) @ (K.T @ beta) / (q * sigma)
    p_val = 1 - scipy.stats.f.cdf(F, q, n-p-1)
    if round(p_val, ndigits=5) == 0:
        p_val = "< 0.000"
    frame = pd.DataFrame({"F": [F], "p-value": p_val})
    return frame

In [33]:
K = np.column_stack(np.array([[0,1,0,0,0,0,0], 
                              [0,0,1,0,0,0,0],
                              [0,0,0,1,0,0,0],
                              [0,0,0,0,1,0,0],
                              [0,0,0,0,0,1,0],
                              [0,0,0,0,0,0,1]]))

- Huomaa, että jokainen vektori (lista) kuvastaa tilannetta, missä kaikki muu paitsi yksi regressio on nolla.
- Ensimmäisessä vektorissa kaikki muut regressiokertoimet paitsi $\, \beta_1 \,$ ovat nollia, toisessa vektorissa kaikki muut paitsi $\, \beta_2 \,$ ovat nollia jne.

In [34]:
K

array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])

<br>

$ H_0 : \beta_1 = \, ... \, = \beta_6 = 0 $ (keskiarvomalli on riittävä selittämään
vasteen arvoissa esiintyvää vaihtelua).

$ H_1 : $ ainakin yksi regressiokerroin $ \neq 0 $.

- Tämän voidaan ajatella testaavan yleisesti regressiomallin merkitsevyyttä.

In [35]:
# Huomaaa, että X, betahat ja sigmahat on määritelty aiemmissa tehtävissä.
F_test(K, X, betahat, sigmahat**2)

,F,p-value
0,16.059733,< 0.000


- Nyt, koska p-arvo on selvästi pienempi kuin yleisesti käytetty $\, \alpha = 0.05, \,$ niin nollahypoteesi voidaan hylätä ja todeta, että keskiarvomalli ei ole paras malli selittämään vasteen arvoissa esiintyvää vaihtelua.

<br>

$ H_0 : \beta_4 = \beta_5 = \beta_6 = 0 \quad $ (regressiokertoimilla $\, \beta_4, \, \beta_5, \, \beta_6 \,$ ei vaikutusta vastemuuttujaan)

In [36]:
K2 = np.column_stack(np.array([[0,0,0,0,1,0,0], 
                               [0,0,0,0,0,1,0],
                               [0,0,0,0,0,0,1]]))

In [37]:
K2

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [38]:
F_test(K2, X, betahat, sigmahat**2)

,F,p-value
0,2.866455,0.037842


- Koska p-arvo on pienempi kuin yleisesti käytetty merkitsevyystaso $\, \alpha = 0.05, \,$ niin nollahypoteesi voidaan hylätä ja todeta, että ainakin yhdellä regressiokertoimista $\, \beta_4, \, \beta_5, \, \beta_6 \,$ on tilastollisesti merkitsevä vaikutus vastemuuttujan arvojen vaihtelun selittämisessä (eli ainakin yksi edellä mainituista regressiokertoimista parantaa mallia).

<br>

$ H_0 : \beta_4 = \beta_6 $

$ H_1 : \beta_4 \neq \beta_6 $

In [39]:
K3 = np.column_stack(np.array([[0,0,0,0,1,0,-1]]))
K3

array([[ 0],
       [ 0],
       [ 0],
       [ 0],
       [ 1],
       [ 0],
       [-1]])

In [40]:
F_test(K3, X, betahat, sigmahat**2)

,F,p-value
0,0.195524,0.658856


- Koska p-arvo on todella suuri, nollahypoteesia ei voida hylätä.
- Siispä voidaan todeta, että regressiokertoimia $\, \beta_4 \,$ ja $\, \beta_5 \,$ vastaavien selittävien muuttujien vaikutukset vastemuuttujaan ovat samankaltaisia eikä molempien regressiokertoimien pitäminen mallissa **välttämättä** paranna mallia.